# Problem Statement:

### The goal of this project is to give people an estimate of how much they need based on their individual health situation. 

# About Dataset

## Context

### The insurance.csv dataset contains 1338 observations (rows) and 7 features (columns). The dataset contains 4 numerical features (age, bmi, children and expenses) and 3 nominal features (sex, smoker and region) that were converted into factors with numerical value designated for each level.

## Acknowledgements

### Insurance.csv file is obtained from the Machine Learning course website (Spring 2017) from Professor Eric Suess at http://www.sci.csueastbay.edu/~esuess/stat6620/#week-6.





###  1. Get the Data


In [ ]:
import pandas as pd
import opendatasets as od
import warnings
warnings.filterwarnings('ignore')

In [ ]:
od.download("https://www.kaggle.com/datasets/noordeen/insurance-premium-prediction")

In [ ]:
data = pd.read_csv("insurance-premium-prediction/insurance.csv")

In [ ]:
data

In [ ]:
data.isna().sum()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data['region'].value_counts()

In [ ]:
data.groupby('sex').count()

In [ ]:
# Preprocessing

import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data.groupby('sex')['expenses'].count().plot.bar();

### Labelized

In [ ]:
le=LabelEncoder()

In [ ]:
cat_col=data.select_dtypes(exclude=['int','float']).columns

In [ ]:
cat_col

In [ ]:
data[cat_col]=data[cat_col].apply(lambda x : le.fit_transform(x))

In [ ]:
data[cat_col]

In [ ]:
data

In [ ]:
sns.kdeplot(data['expenses']);

In [11]:
sns.kdeplot(data['age']);

NameError: name 'sns' is not defined

In [ ]:

plt.figure(figsize=(12,10))
sns.countplot(data['sex']);

In [12]:
plt.figure(figsize=(12,10))
sns.scatterplot(y=data['age'],x=data['expenses'],hue=data['sex'])
plt.show()

NameError: name 'plt' is not defined

In [ ]:
data.corr()

In [13]:
plt.figure(figsize=(12,10))
sns.heatmap(data.corr(),annot=True,cmap='viridis')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# drop feature region  as it has negative co-relation with expenses
data.drop(columns="region",inplace=True)

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(data.corr(),annot=True,cmap='viridis')
plt.show()

In [14]:
plt.figure(figsize=(12,10))
sns.kdeplot(data['expenses'],shade=True);
plt.axvline(data['expenses'].mean(),color='red',linewidth=2)
plt.axvline(data['expenses'].median(),color='green',linewidth=2)
plt.show()

NameError: name 'plt' is not defined

In [15]:
plt.figure(figsize=(35,35))
sns.pairplot(data,hue='sex', size=4)
plt.show()

NameError: name 'plt' is not defined

In [ ]:

data.groupby(by='sex').corr()

In [16]:
plt.figure(figsize=(12,10))
sns.heatmap(data.groupby(by='sex').corr(),annot=True,cmap='viridis')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X = data.drop(columns='expenses')
y=data['expenses']
num_col=X.select_dtypes(include=['int','float']).columns

## Scalized

In [32]:
sc=StandardScaler()
X[num_col]=sc.fit_transform(X[num_col])

In [6]:
x

NameError: name 'x' is not defined

## Spliting the X & y

In [33]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

## HyperTuning model and param

In [34]:
model_parameter ={

"Linear_Regression" :{
                        "model" : LinearRegression(),
                        "param" : {}
                    },

"Decision_Tree_Regressor" :{
                        "model" : DecisionTreeRegressor(),
                        "param" : {
                                   'splitter' : ["best"],
                                   'ccp_alpha':[0.1,0.0,0.3]
                                  }
                            },

"SVR" : {
                        "model" : SVR(),
                        "param" : {}
       },

"KNeighbors_Classifier_Regressor" : {
                        "model" : KNeighborsRegressor(),
                        "param" : {'n_neighbors' : [4,5,6],
                                  'weights' : ['uniform', 'distance']
                                  }
},
                       

"Random_Forest_Regressor" :{
                    "model" : RandomForestRegressor(),
                    "param" : {'criterion' : ["squared_error", "absolute_error"],
                              'n_estimators':[100,85,90]
                              }
                      },
 "Bagging_Regressor" :{
                    "model" : BaggingRegressor(base_estimator=SVR(),random_state=0),
                    "param" : {'n_estimators':[10,100]}
                      },
    


 "AdaBoost_Regressor" :{
                    "model" : AdaBoostRegressor(),
                    "param" : {}
                      },

"Gradient_Boosting_Regressor" :{
                    "model" : GradientBoostingRegressor(),
                    "param" : {}
                      },

"XGB_Regressor" :{
                    "model" : XGBRegressor(),
                    "param" : {}
                      },
}

In [35]:
Report = []
for model_name ,model_param in model_parameter.items():
    gs = GridSearchCV(model_param['model'],model_param['param'],cv = 5,return_train_score=False)
    gs.fit(X_train,y_train)
    Report.append({'Model': model_name,
                 'Best_Score' : gs.best_score_,
                 'Best_Parameter' : gs.best_params_})

Model_Report = pd.DataFrame(Report,columns=['Model','Best_Score','Best_Parameter'])

In [36]:
Model_Report.sort_values(by='Best_Score',ascending=False)

,Model,Best_Score,Best_Parameter
7,Gradient_Boosting_Regressor,0.851847,{}
4,Random_Forest_Regressor,0.825564,"{'criterion': 'squared_error', 'n_estimators':..."
3,KNeighbors_Classifier_Regressor,0.820967,"{'n_neighbors': 6, 'weights': 'uniform'}"
6,AdaBoost_Regressor,0.806599,{}
8,XGB_Regressor,0.795223,{}
0,Linear_Regression,0.748869,{}
1,Decision_Tree_Regressor,0.708432,"{'ccp_alpha': 0.3, 'splitter': 'best'}"
5,Bagging_Regressor,-0.105179,{'n_estimators': 10}
2,SVR,-0.107787,{}


In [37]:
gs.predict(X_test)

array([10066.824  , 27609.195  , 15552.384  ,  7782.336  ,  9573.726  ,
        4417.5464 , 13533.211  ,  6039.5586 ,  9264.508  , 17785.686  ,
       17410.506  ,  1786.154  ,  8725.821  ,  4305.0024 ,  9353.103  ,
       12651.421  , 33767.5    ,  3450.488  , 13861.167  ,  4675.5947 ,
       14135.426  ,  1192.298  , 29705.465  ,  1210.4293 ,  6311.2495 ,
       20173.506  ,  9256.528  , 16394.021  ,  6899.074  , 12555.286  ,
        9931.486  , 10071.837  ,   710.7342 , 28097.227  ,  4232.803  ,
        7742.23   ,  5802.721  ,  6486.261  , 20191.076  , 20886.248  ,
       11465.281  ,  6284.0947 ,  9532.06   , 11724.107  , 39888.258  ,
       41623.914  , 21308.146  , 10839.9375 , 19867.924  ,  2006.0054 ,
       23056.588  ,  6817.743  , 14682.619  ,  9533.065  , 18025.053  ,
        6602.4863 , 15576.417  ,  3112.492  , 41520.27   ,  4628.109  ,
        9638.439  ,  2023.4198 , 14976.197  , 11302.815  ,  6056.125  ,
        8514.4375 ,  3879.0503 , 37881.645  ,  5191.667  ,  2047

In [38]:
import joblib
joblib.dump(gs,"model.gbr")
joblib.dump(sc,"scaler.sc")

['scaler.sc']